# Tensorboard 사용법

참고 사이트: https://www.tensorflow.org/tensorboard/get_started

In [56]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Reshape, BatchNormalization, Input, MaxPooling2D, Dense, Activation,\
    Conv2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import sys
import datetime
import numpy as np

In [2]:
print('Tensorflow version : ', tf.__version__)
print('System version : ', sys.version_info)

Tensorflow version :  2.3.0
System version :  sys.version_info(major=3, minor=7, micro=11, releaselevel='final', serial=0)


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

w, h = 28, 28

x_train = x_train.reshape(-1, w, h, 1)
x_test = x_test.reshape(-1, w, h, 1)

x_train = x_train / 255.
x_test = x_test / 255.

In [4]:
y_train.shape

(60000, 10)

In [5]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [6]:
x_train.shape

(60000, 28, 28, 1)

In [7]:
def build(data_shape):
    inputs = Input(data_shape)

    x = Conv2D(32, (3, 3), padding='same')(inputs)
    x = Activation('relu')(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = Activation('relu')(x)

    x = Flatten()(x)
    x = Dense(10)(x)
    x = Activation('sigmoid')(x)

    model = Model(inputs = inputs, outputs = x)


    return model

In [8]:
model = build(x_train.shape[1:])

In [9]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 10)               

In [10]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics = ['acc'])

In [15]:
EPOCHS = 5
BATCH_SIZE = 256

## tensorboard with Keras Model.fit()

In [11]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [12]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [13]:
result = model.fit(x_train, y_train,epochs = EPOCHS, batch_size = BATCH_SIZE, validation_data = (x_test, y_test), verbose = 1, callbacks=[tensorboard_callback])

Epoch 1/10
  1/235 [..............................] - ETA: 0s - loss: 0.6906 - acc: 0.1172WARNING:tensorflow:From C:\ProgramData\Anaconda3\envs\yolos\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
235/235 [==============================] - 3s 14ms/step - loss: 0.0722 - acc: 0.9082 - val_loss: 0.0191 - val_acc: 0.9780
Epoch 2/10
235/235 [==============================] - 3s 12ms/step - loss: 0.0173 - acc: 0.9805 - val_loss: 0.0144 - val_acc: 0.9829
Epoch 3/10
235/235 [==============================] - 3s 12ms/step - loss: 0.0126 - acc: 0.9861 - val_loss: 0.0116 - val_acc: 0.9877
Epoch 4/10
235/235 [==============================] - 3s 12ms/step - loss: 0.0099 - acc: 0.9896 - val_loss: 0.0107 - val_acc: 0.9875
Epoch 5/10
235/235 [==============================] - 3s 12ms/step - loss: 0.0080 - acc: 0.9922 

In [14]:
!tensorboard --logdir logs/fit

^C


## tf.GradientTape with tf.summary()

In [95]:
loss_object = tf.keras.losses.CategoricalCrossentropy() # one-hot vector에 사용

train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.CategoricalAccuracy('train_accuracy')
optimizer = tf.keras.optimizers.Adam()

In [100]:
log_dir = "logs/fit2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_summary_writer = tf.summary.create_file_writer(log_dir)

In [101]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_accuracy(y_train, predictions)
    train_loss(loss)
        
    return loss

In [102]:
batchs = x_train.shape[0] // BATCH_SIZE

In [103]:
batchs

234

In [104]:
model = build(x_train.shape[1:])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics = ['acc'])

In [105]:
for epoch in range(EPOCHS):
    epoch_loss = []
    for i in range(batchs):
        losses = train_step(model, optimizer, x_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE], y_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE])
        epoch_loss.append(losses)
        
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
            
            
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print (template.format(epoch+1,np.mean(epoch_loss), train_accuracy.result()*100))

Epoch 1, Loss: 0.2717607617378235, Accuracy: 92.45793151855469
Epoch 2, Loss: 0.07162335515022278, Accuracy: 95.15391540527344
Epoch 3, Loss: 0.04829372465610504, Accuracy: 96.29351806640625
Epoch 4, Loss: 0.03638845309615135, Accuracy: 96.95011901855469
Epoch 5, Loss: 0.028411708772182465, Accuracy: 97.39649963378906


In [107]:
!tensorboard --logdir logs/fit2 --port=9000

^C
